# Predict $\gamma$ with 1D CNNs

In [15]:
import tensorflow as tf

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv1D, BatchNormalization, ReLU

## Custom CNN

In [16]:
def ResBlock1D(in_c, out_c, kernel_size, stride1=1, stride2=1, padding='same', bias=False, downsample=None) :
    '''
    Defines a residual block for a 1D CNN
    '''
    
    # initialize input
    X = Input(shape=(None, in_c))
    inp = X
    
    # conv1
    out = Conv1D(out_c, kernel_size, strides=stride1, padding=padding, use_bias=bias)(inp)
    out = BatchNormalization()(out)
    out = ReLU()(out)
    
    # conv2
    out = Conv1D(out_c, kernel_size, strides=stride2, padding=padding, use_bias=bias)(out)
    out = BatchNormalization()(out)
    
    # downsample if specified
    if downsample is not None :
        inp = downsample(X)
        
    # residual connection
    out += X
    out = ReLU()(out)
    
    return Model(inp, out)

In [17]:
rb0 = ResBlock1D(32, 32, 3)
rb0.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, None, 32)]   0           []                               
                                                                                                  
 conv1d_4 (Conv1D)              (None, None, 32)     3072        ['input_4[0][0]']                
                                                                                                  
 batch_normalization_2 (BatchNo  (None, None, 32)    128         ['conv1d_4[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 re_lu_2 (ReLU)                 (None, None, 32)     0           ['batch_normalization_2[0][0]